- pokemon_data.csv - 포켓몬의 다양한 정보를 가진 원본 csv 파일
- description.csv - GPT를 이용해 포켓몬 당 설명 3개 저장한 csv 파일

1. pokemon_data.csv의 name 열은 lower case로 저장되어 있음.
2. description.csv와 가끔 명칭이 다른 포켓몬이 있음.

위 두가지를 전처리

In [7]:
import csv

# input.txt 파일에서 읽기
with open('descriptions.txt', 'r', encoding='utf-8') as infile:
    lines = infile.readlines()

# 결과를 저장할 리스트
data = []

for line in lines:
    line = line.strip()
    if ':' in line:  # en-dash (–) 사용 주의
        name, description = line.split(':', 1)
        name = name.strip()
        description = description.strip()
        data.append((name, description))

# CSV 파일로 저장
with open('output.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['name', 'description'])  # 헤더 쓰기
    writer.writerows(data)

print('변환 완료: output.csv')

변환 완료: output.csv


In [8]:
import pandas as pd

df_full = pd.read_csv('/home/pys/2025-1-Web3-cuketmon/AI/filtered_file.csv')
df_desc = pd.read_csv('output.csv')

df_desc['name'] = df_desc['name'].astype(str).str.lower()

df_desc.to_csv('description_lowercase.csv', index=False)

print("✅ name 컬럼을 소문자로 변환하여 description_lowercase.csv로 저장 완료!")

# name 컬럼만 추출 후 공백 제거
names_full = set(df_full['name'].astype(str).str.strip())
names_desc = set(df_desc['name'].astype(str).str.strip())

# 차이 계산
missing_in_desc = names_full - names_desc

# 결과 출력
print(f"✅ output.csv에 없는 name 수: {len(missing_in_desc)}")
for name in sorted(missing_in_desc):
    print(f"{name}")

✅ name 컬럼을 소문자로 변환하여 description_lowercase.csv로 저장 완료!
✅ output.csv에 없는 name 수: 0


descrition.csv는 name에 3개의 열이 중복됨.
name을 하나로 만들고 description 열을 3개 만들어서 전처리

In [9]:
import pandas as pd

# description.csv 불러오기
df_desc = pd.read_csv('description_lowercase.csv')
df_desc['name'] = df_desc['name'].str.lower()

# 각 name마다 description을 리스트로 묶음
grouped = df_desc.groupby('name')['description'].apply(list)

# 리스트를 개별 열로 나눔
max_desc = 1  # 최대 설명 개수
df_wide = grouped.apply(lambda x: x[:max_desc] + [""] * (max_desc - len(x))).apply(pd.Series)
df_wide.columns = [f'description{i+1}' for i in range(max_desc)]
df_wide = df_wide.reset_index()

# 저장
df_wide.to_csv('description_wide.csv', index=False)
print("✅ description_wide.csv 생성 완료 (description1~3 열 포함)")

✅ description_wide.csv 생성 완료 (description1~3 열 포함)


설명을 랜덤으로 pokemon_data.csv에 매핑

In [10]:
import random
import pandas as pd

# 1. 데이터 로드
df_pokemon = pd.read_csv('/home/pys/2025-1-Web3-cuketmon/AI/filtered_file.csv')
df_desc = pd.read_csv('description_wide.csv')  # name, description1~3

# 2. description1만 사용
desc_dict = df_desc.set_index('name')[['description1']].to_dict(orient='index')

# 3. pokemon 데이터에 description1과 type을 합쳐서 prompt 생성
def create_prompt(name, pokemon_type):
    # description1 가져오기
    entry = desc_dict.get(name)
    if entry:
        description = entry['description1']
        # prompt 생성
        return f"a poketmon, type is {pokemon_type}, {description}"
    return ""

# 4. 각 행에 대해 prompt 생성
df_pokemon['description'] = df_pokemon.apply(
    lambda row: create_prompt(row['name'], row['type']),
    axis=1
)

# 5. 결과 저장
df_pokemon.to_csv('pokemon_data_with_prompt.csv', index=False)
print("✅ 각 행마다 prompt 생성 완료!")


✅ 각 행마다 prompt 생성 완료!


In [11]:
import pandas as pd
import json
import os

# CSV 경로와 출력 경로 설정
csv_path = "pokemon_data_with_prompt.csv"
output_path = "/mnt/nas-drive/pys/cuketmon/datasets/pokemon_images/metadata.jsonl"

# CSV 로드
df = pd.read_csv(csv_path)

# 필요한 열만 사용
df = df[["image_filename", "description"]].dropna()

# JSONL 파일로 저장
with open(output_path, "w", encoding="utf-8") as f:
    for _, row in df.iterrows():
        entry = {
            "file_name": "/mnt/nas-drive/pys/cuketmon/datasets/pokemon_images/"+row["image_filename"],
            "text": row["description"]
        }
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

print(f"metadata.jsonl created with {len(df)} entries.")

metadata.jsonl created with 3285 entries.


In [12]:
import pandas as pd

# 원본 CSV 파일 경로와 출력할 CSV 파일 경로를 지정합니다.
input_csv = "/mnt/nas-drive/pys/cuketmon/datasets/pokemon_data.csv"       # 예: 입력 파일 이름
output_csv = "/mnt/nas-drive/pys/cuketmon/datasets/pokemon_data.csv"     # 예: 정렬된 결과를 저장할 파일 이름

# CSV 파일 읽기
df = pd.read_csv(input_csv)

# 'image_filename' 열을 기준으로 정렬 (오름차순 정렬)
df_sorted = df.sort_values(by="image_filename")

# 정렬 결과를 새로운 CSV 파일로 저장 (인덱스는 저장하지 않음)
df_sorted.to_csv(output_csv, index=False)

print("CSV 파일이 'image_filename' 기준으로 정렬되어", output_csv, "파일로 저장되었습니다.")


CSV 파일이 'image_filename' 기준으로 정렬되어 /mnt/nas-drive/pys/cuketmon/datasets/pokemon_data.csv 파일로 저장되었습니다.
